In [1]:
import numpy as np
import cv2
from Detector_COV.detection_class import detector_class

from scripts.preprocessor_functions import remove_vignette

import os
import json
from skimage.morphology import disk
import time


In [2]:
SET_NAME = "Set_7"
MATERIAL = "Graphene"
SET_DIRS = r"C:\Users\Uslu.INSTITUT2B\Desktop\Annotated_Images"

SET_DIR = os.path.join(SET_DIRS, MATERIAL,SET_NAME)
IMAGE_DIR = os.path.join(SET_DIR, "Images")
FLATFIELD = os.path.join(SET_DIR, "flatfield.png")
PARAMETER_PATH = os.path.join(SET_DIR, "gaussian_mixture_model_contrast_data.json")
DETECTED_SAVE_DIR = os.path.join(SET_DIR, "Detected_Images")

In [ ]:
with open(PARAMETER_PATH) as f:
    contrast_params = json.load(f)

SIZE_THRESHOLD = 400
MAGNIFICATION = 20
STANDARD_DEVIATION_THRESHOLD = 5
COVARIANCE_SCALING_FACTORS = [1, 1, 1]
USED_CHANNELS = [0, 1, 2]


myDetector = detector_class(
    contrast_dict=contrast_params,
    size_threshold=SIZE_THRESHOLD,
    magnification=MAGNIFICATION,
    standard_deviation_threshold=STANDARD_DEVIATION_THRESHOLD,
    used_channels=USED_CHANNELS,
    covariance_scaling_factors=COVARIANCE_SCALING_FACTORS,
)

In [3]:
FF = cv2.imread(FLATFIELD)

IMAGE_NAMES = os.listdir(IMAGE_DIR)

current_image_number = 0
current_flake_number = 0
for image_name in IMAGE_NAMES:
    image = cv2.imread(os.path.join(IMAGE_DIR, image_name))

    image = remove_vignette(image, FF)

    start = time.time()
    detected_flakes = myDetector.detect_flakes(image)
    print(
        f"Time : {np.round(time.time() - start,3)}s | Detected flakes : {len(detected_flakes)} Name : {image_name}"
    )
    # Operation on the flakes
    if len(detected_flakes) != 0:
        current_image_number += 1
        all_flake_mask = np.zeros(image.shape[:2], dtype=np.uint8)

        # Operate on each Flake
        for flake in detected_flakes:
            current_flake_number += 1

            flake_mask = flake["mask"]
            flake_thickness = flake["layer"]

            flake_thickness_dir = os.path.join(SET_DIR, f"Masks_{flake_thickness}")
            if not os.path.exists(flake_thickness_dir):
                os.makedirs(flake_thickness_dir)

            flake_mask_path = os.path.join(
                flake_thickness_dir, f"MASK_{current_flake_number}_{image_name}"
            )

            cv2.imwrite(flake_mask_path, flake_mask)

            all_flake_mask = cv2.bitwise_or(all_flake_mask, flake_mask, all_flake_mask)

            # # Extract some data from the Dict
            (x, y) = flake["position_bbox"]
            w = flake["width_bbox"]
            h = flake["height_bbox"]
            # cv2.rectangle(
            #     image,
            #     (x - 20, y - 20),
            #     (x + w + 20, y + h + 20),
            #     color=[0, 0, 0],
            #     thickness=2,
            # )

            # Dilate the flake outline and get the gradient to get a nice border
            # outline_flake = cv2.dilate(flake["mask"], disk(2))
            outline_flake = cv2.morphologyEx(
                flake_mask,
                cv2.MORPH_GRADIENT,
                disk(1),
            )

            # Draw this border on the image
            image[outline_flake != 0] = [0, 0, 255]

            cv2.putText(
                image,
                f"{flake_thickness}",
                (x + w // 2, y + h // 2),
                cv2.FONT_HERSHEY_SIMPLEX,
                thickness=2,
                fontScale=2,
                color=(0, 0, 0),
            )
            #####

        # Now save the Image to its new Home
        cv2.imwrite(
            os.path.join(DETECTED_SAVE_DIR, f"Detected_{image_name}"),
            image,
        )


Time : 0.967s | Detected flakes : 1 Name : 01cf2db5-aba8-451f-a8ea-95736a47f7b7.png
Time : 0.274s | Detected flakes : 1 Name : 030f9d7c-a78a-44e8-8d4b-54894a3cfb7c.png
Time : 0.271s | Detected flakes : 1 Name : 03b36349-68ec-42e6-b256-5ba3b231d4e6.png
Time : 0.265s | Detected flakes : 1 Name : 0672386c-0125-44be-983e-64d815caeb62.png
Time : 0.31s | Detected flakes : 1 Name : 0a20f325-0fa8-49f8-8b99-7defebc1c35c.png
Time : 0.286s | Detected flakes : 1 Name : 0c2f2365-d75b-461b-9345-6506e9264aae.png
Time : 0.343s | Detected flakes : 3 Name : 0c3bdecb-0cbf-4ed2-88f0-e3b5fbe3206c.png
Time : 0.277s | Detected flakes : 1 Name : 0f64280b-de95-4165-a223-526348233e02.png
Time : 0.262s | Detected flakes : 1 Name : 11fad192-42e4-4a88-842e-7b4a3823bcaf.png
Time : 0.273s | Detected flakes : 1 Name : 12596fc1-c1ce-41ab-8e84-91275369f916.png
Time : 0.481s | Detected flakes : 0 Name : 150f9165-90e5-4fac-b6b6-70dffb548e12.png
Time : 0.267s | Detected flakes : 1 Name : 1527e4d6-21bc-49af-9a6c-b287de937e